# Advection in Spherical coordinates
In this notebook, we will solve the advection equation in spherical coordinates using the finite volume method. The notebook serves as the first attempt at testing the implementation of advection and diffusion terms in `PyFVTool`, and might be removed in the future.

In [1]:
import sys

# Use a raw string for the path
sys.path.append(r'C:\projects\git\PyFVTool\src')

# Print sys.path to verify the path is added
print(sys.path)




['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.1264.0_x64__qbz5n2kfra8p0', 'c:\\Users\\aliak\\venvs\\mygeo', '', 'c:\\Users\\aliak\\venvs\\mygeo\\Lib\\site-packages', 'c:\\Users\\aliak\\venvs\\mygeo\\Lib\\site-packages\\win32', 'c:\\Users\\aliak\\venvs\\mygeo\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\aliak\\venvs\\mygeo\\Lib\\site-packages\\Pythonwin', 'C:\\projects\\git\\PyFVTool\\src']
Error importing module: cannot import name 'module_name' from 'pyfvtool' (C:\projects\git\PyFVTool\src\pyfvtool\__init__.py)


In [4]:
from pyfvtool.mesh import SphericalGrid3D
import numpy as np
# Create a spherical grid with a 10 km thickness
r_earth = 6371.0e3 # Earth radius in meters
atm_layer_thickness = 10.0e3 # 10 km thickness
m = SphericalGrid3D(10, 100, 100, atm_layer_thickness, 2*np.pi, 2*np.pi) 
# shift the grid to the Earth surface
